## SQLite intro

- sqlite3 reference: https://docs.python.org/3.6/library/sqlite3.html
- sqlite cli reference: https://sqlite.org/cli.html (useful to explore DBs and tables)

In [1]:
import sqlite3

Database connection:
- ':memory:' creates a temporary database in the computer memory (RAM, faster access)
- path: creates/open a database on the filesystem

In [2]:
conn = sqlite3.connect(':memory:')

In [3]:
# Get connection cursor -> used to execute sql queries
cur = conn.cursor()

In [4]:
query = """
CREATE TABLE People (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  firstname TEXT,
  lastname TEXT
)
"""

In [5]:
cur.execute(query)

In [6]:
conn.commit() #commit() needed to apply all the pending operations

In [7]:
cur.execute('SELECT * from People')
cur.fetchall()

[]

In [8]:
personA = ('mario', 'rossi')

# insertion in a table, argument can be a tuple
cur.execute('INSERT INTO People (firstname, lastname) VALUES (?,?)', personA)

conn.commit()

In [9]:
cur.execute('SELECT * from People')
cur.fetchall()

[(1, 'mario', 'rossi')]

In [10]:
people = [('paolo', 'neri'), ('jhon', 'doe')]

#inserting multiple values at once with executemany() and list of tuples
cur.executemany('INSERT INTO People (firstname, lastname) VALUES (?, ?)', people)

conn.commit()

In [11]:
# hardcoded values (or string concatenation) -> not a best practice
cur.execute('INSERT INTO People (firstname, lastname) VALUES ("luca", "neri")')
conn.commit()

In [12]:
# a SELECT query returns a list of rows (as tuples)
lastname = 'neri'
for row in cur.execute(f'SELECT * from People WHERE lastname="{lastname}"'):
    print(row)

(2, 'paolo', 'neri')
(4, 'luca', 'neri')


In [13]:
# deleting a row
cur.execute('DELETE FROM People WHERE firstname="luca" AND lastname="neri"')
conn.commit()

In [14]:
cur.execute('SELECT * from People')
cur.fetchall()

[(1, 'mario', 'rossi'), (2, 'paolo', 'neri'), (3, 'jhon', 'doe')]

In [15]:
cur.execute('INSERT INTO People (firstname, lastname) VALUES ("luca", "neri")')
conn.commit()

In [17]:
# query results are stored inside the cursos object.
# we can use fetchone[fetchall] to get one first row[list of rows]
# see fetchone(), fetchall(), fetchmany(n)
cur.execute('SELECT * from People')
cur.fetchone()

(1, 'mario', 'rossi')

In [18]:
# multiple fetchone() calls -> get the next result
cur.fetchone()

(2, 'paolo', 'neri')

In [19]:
# trying to insert data into a non-existing table -> exception raised
try:
    cur.execute('INSERT INTO Animals (firstname, lastname) VALUES ("cane", "cane")')
except Exception as e:
    print(e)
    print(type(e))

no such table: Animals
<class 'sqlite3.OperationalError'>


In [29]:
# close connection to database
conn.close()

### JSON dataset to SQLite database

In [24]:
!unzip tweet.zip

Archive:  tweet.zip
   creating: SampleFromData/
  inflating: SampleFromData/1000002204978372608  
  inflating: SampleFromData/1000001566110339074  
  inflating: SampleFromData/1000001778904166401  
  inflating: SampleFromData/1000001970009182208  
  inflating: SampleFromData/1000001726852759552  
  inflating: SampleFromData/1000001524339224577  
  inflating: SampleFromData/1000001821858062336  
  inflating: SampleFromData/1000002148376236032  
  inflating: SampleFromData/1000001661606203392  
  inflating: SampleFromData/1000001570703069188  
  inflating: SampleFromData/1000001759379705858  
  inflating: SampleFromData/1000001560297005056  
  inflating: SampleFromData/1000002204089102336  
  inflating: SampleFromData/1000002015702016000  
  inflating: SampleFromData/1000001530232299520  
  inflating: SampleFromData/1000001911100182528  
  inflating: SampleFromData/1000002164020957184  
  inflating: SampleFromData/1000002121889087491  
  inflating: SampleFromData/1000002030675611648  
 

In [25]:
!head SampleFromData/1000002204978372608

{"usernameTweet": "CervoVolante", "ID": "1000002204978372608", "text": "Gli state dando i compiti per gli esami ?????", "url": "/CervoVolante/status/1000002204978372608", "nbr_retweet": 0, "nbr_favorite": 0, "nbr_reply": 0, "datetime": "2018-05-25 09:14:39", "is_reply": true, "is_retweet": false, "user_id": "190688480"}

In [26]:
import glob
import json

def read_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

In [30]:
# create a new file database to store the tweets
conn = sqlite3.connect('tweet.sqlite3')
cur = conn.cursor()

In [31]:
# Table structure inferred from json files
query = """
CREATE TABLE Tweets (
    usernameTweet TEXT,
    ID TEXT PRIMARY KEY UNIQUE,
    text TEXT,
    url TEXT,
    nbr_retweet INTEGER,
    nbr_favorite INTEGER,
    nbr_reply INTEGER,
    datetime TEXT,
    is_reply BOOLEAN,
    is_retweet BOOLEAN,
    user_id TEXT
)
"""
cur.execute(query)
conn.commit()

In [32]:
# If we forgot a column, we can add it to the table with the ALTER statement
cur.execute('ALTER TABLE Tweets ADD has_media BOOLEAN')
conn.commit()

In [33]:
# Media tables, to store the (optional) media URIs which can be found in a tweet
query = """
CREATE TABLE Medias(
    media_id INTEGER PRIMARY KEY AUTOINCREMENT,
    tweet_id TEXT,
    media_uri TEXT,
    FOREIGN KEY(tweet_id) REFERENCES Tweets(ID)
)
"""
cur.execute(query)
conn.commit()

In [34]:
# Iterate through all of the json files
for filename in glob.glob('SampleFromData/*'):
    data = read_json(filename)
    medias = []
    
    # if the tweet has medias:
    # - add all media entries to the list (medias)
    # - delete medias object from the json data (we can infer this by joining Tweets
    #                                            and Medias table)
    if 'medias' in data:
        for media in data['medias']:
            medias.append((data['ID'], media))
        
        del data['medias']
        
    keys = data.keys()
    
    placeholders = ('?,'*len(keys))[:-1]
    keys = ', '.join(keys)
    values = data.values()
    
    try:
        cur.execute('INSERT INTO Tweets ({0}) VALUES ({1})'.format(keys, placeholders), list(values))
        if len(medias) > 0:
            cur.executemany('INSERT INTO Medias (tweet_id, media_uri) VALUES (?, ?)', medias)
    
    # prevent unique index constraint fail (ID column) if we run this cell
    # more than once
    except sqlite3.IntegrityError as r: 
        pass
    except Exception as e:
        print(data)
        raise e

In [35]:
# finalize pending inserts (from cell above)
conn.commit()

In [38]:
cur.execute('SELECT * FROM Tweets LIMIT 10')
cur.fetchone()

('GiuliaCrivelli',
 '1000001083509559296',
 'grazie Presidente!!!!!',
 '/GiuliaCrivelli/status/1000001083509559296',
 0,
 1,
 1,
 '2018-05-25 09:10:12',
 1,
 0,
 '586429781',
 None)

In [41]:
cur.execute('SELECT * FROM Medias')
cur.fetchall()

[(1, '1000000751379283969', 'https://t.co/2NIWx9bUma'),
 (2, '1000000610039627783', 'https://t.co/YgE3tDs1LM'),
 (3, '1000001089926828033', 'https://t.co/IkJe1WVsEp'),
 (4, '1000001211880411136', 'https://t.co/aaXzPBs6Yx'),
 (5, '1000001118070550528', 'https://t.co/vf71oZtovx'),
 (6, '1000000791694934016', 'https://t.co/CV77QNnYMx'),
 (7, '1000001205190496256', 'https://t.co/hGXBHlcsAt'),
 (8, '1000001726852759552', 'https://t.co/lIAiqCmXfc'),
 (9, '1000000566821638144', 'https://t.co/AbPxCrtVck')]

In [42]:
conn.close()